In [1]:
import numpy as np
import matplotlib.pyplot as plt

from training.data_augmentation.model import MCTS_Augment



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [2]:
data = np.array([1 / (each + 1) for each in range(1, 1001)])
noise = np.random.normal(0.1, 0.01, size=(1000,))
score = data + noise

# plt.figure(figsize=(20, 10))
# plt.plot(range(1000), score)

In [3]:
search_augment = MCTS_Augment((64, 320, 240), None)

In [4]:
for each_index, each in enumerate(score):
    out_str, _ = search_augment.update_epoch(each)
    search_augment.build_callable()
    print(each_index, out_str, "\n", search_augment.traverse_path)

0 
uniform sampling at layer 1, tree have 22 nodes
uniform sampling at layer 2, tree have 448 nodes
uniform sampling at layer 3, tree have 8412 nodes
uniform sampling at layer 4, tree have 144984 nodes 
 [14, 8, 12, 16]
1 
uniform sampling at layer 1, tree have 22 nodes
uniform sampling at layer 2, tree have 448 nodes
uniform sampling at layer 3, tree have 8412 nodes
uniform sampling at layer 4, tree have 144984 nodes 
 [20, 18, 2, 17]
2 
uniform sampling at layer 1, tree have 22 nodes
uniform sampling at layer 2, tree have 448 nodes
uniform sampling at layer 3, tree have 8412 nodes
uniform sampling at layer 4, tree have 144984 nodes 
 [15, 16, 3, 14]
3 
uniform sampling at layer 1, tree have 22 nodes
uniform sampling at layer 2, tree have 448 nodes
uniform sampling at layer 3, tree have 8412 nodes
uniform sampling at layer 4, tree have 144984 nodes 
 [18, 18, 0, 3]
4 
uniform sampling at layer 1, tree have 22 nodes
uniform sampling at layer 2, tree have 448 nodes
uniform sampling at l

TypeError: cannot pickle 'SwigPyObject' object

In [ ]:
visited_times_threshold_to_uct_sample = [4, 2, 1, 1]
tree_layer_population_count = []
new_traverse_path = []
node_layer_population = search_augment.root.children
tree_layer_population = [search_augment.root]
for layer_index in range(search_augment.depth):
    # uct need all the node of the TREE from this layer
    tree_layer_population = search_augment.find_all_children_from_list(tree_layer_population)
    tree_layer_population_count.append(len(tree_layer_population))
    # base on section 3.4 P3, only sample when average visited time of children is larger than threshold
    mean_visited_times = sum(map(lambda each: each.visited_times, node_layer_population)) / len(
        node_layer_population
    )
    if mean_visited_times > visited_times_threshold_to_uct_sample[layer_index]:
        raw = [
            search_augment.calculate_uct(each, tree_layer_population) / search_augment.eq6_tau
            for each in node_layer_population
        ]
        exps = np.exp(raw)
        prob = exps / exps.sum()
        new_index = np.random.choice(np.arange(len(node_layer_population)), p=prob)
        out_str += (
            f"    uct sampling at layer {layer_index+1}, tree have {len(tree_layer_population)} nodes\n"
        )
    else:
        new_index = np.random.choice(np.arange(len(node_layer_population)))
        out_str += (
            f"uniform sampling at layer {layer_index+1}, tree have {len(tree_layer_population)} nodes\n"
        )